In [92]:
import hnswlib
import numpy as np 
import struct
import heapq

In [93]:
from pathlib import Path
index_path='../indexes/SIFT1M_index.bin'#.format(dbname)
data = Path(index_path).read_bytes()
len(data)

660564936

The parameters in the index header, stored in small endian
uint64_t offsetLevel0_; // 0:8
uint64_t max_elements_; // 8:16
uint64_t cur_element_count; // 16:24
uint64_t size_data_per_element_; // 24:32
uint64_t label_offset_; // 32:40
uint64_t offsetData_; // 40:48
uint32_t maxlevel_; // 48:52
uint32_t enterpoint_node_; // 52:56
uint64_t maxM_; // 56:64
uint64_t maxM0_; // 64:72
uint64_t M_; // 72:80
double mult_; // 80:88
uint64_t ef_construction_; // 88:96

Results I got from C++ on SIFT1M:

Index file size: 660564936
offsetLevel0_: 0
max_elements_: 1000000
cur_element_count: 1000000
size_data_per_element_: 652
label_offset_: 644
offsetData_: 132
maxlevel_: 5
enterpoint_node_: 572337
maxM_: 16
maxM0_: 32
M_: 16
mult_: 0.360674
ef_construction_: 128

size_data_per_element_ 652 = sizeVec (128 * 4) + maxM0_ * link_num (4) size_link_ID (32 * 4) + vec_ID (8) 

In [94]:
# maxlevel_
offsetLevel0_ = int.from_bytes(data[0:8], byteorder='little', signed=False)
max_elements_ = int.from_bytes(data[8:16], byteorder='little', signed=False)
cur_element_count = int.from_bytes(data[16:24], byteorder='little', signed=False)
size_data_per_element_ = int.from_bytes(data[24:32], byteorder='little', signed=False)
label_offset_ = int.from_bytes(data[32:40], byteorder='little', signed=False)
offsetData_ = int.from_bytes(data[40:48], byteorder='little', signed=False)
maxlevel_ = int.from_bytes(data[48:52], byteorder='little', signed=False)
enterpoint_node_ = int.from_bytes(data[52:56], byteorder='little', signed=False)
maxM_ = int.from_bytes(data[56:64], byteorder='little', signed=False)
maxM0_ = int.from_bytes(data[64:72], byteorder='little', signed=False)
M_ = int.from_bytes(data[72:80], byteorder='little', signed=False)
ef_construction_ = int.from_bytes(data[88:96], byteorder='little', signed=False)
# mult_ = float.from_bytes(data[80:88], byteorder='little')

print("offsetLevel0_", offsetLevel0_)
print("max_elements_", max_elements_)
print("cur_element_count", cur_element_count)
print("size_data_per_element_", size_data_per_element_)
print("label_offset_", label_offset_)
print("offsetData_", offsetData_)
print("maxlevel_", maxlevel_)
print("enterpoint_node_", enterpoint_node_)
print("maxM_", maxM_)
print("maxM0_", maxM0_)
print("M_", M_)
print("ef_construction_", ef_construction_)

offsetLevel0_ 0
max_elements_ 1000000
cur_element_count 1000000
size_data_per_element_ 652
label_offset_ 644
offsetData_ 132
maxlevel_ 5
enterpoint_node_ 572337
maxM_ 16
maxM0_ 32
M_ 16
ef_construction_ 128


In [95]:
print("totol bytes: {}".format(len(data)))
print("ground layer bytes: {}".format(max_elements_ * size_data_per_element_))
print("rest layers: {}".format(len(data) - max_elements_ * size_data_per_element_))

totol bytes: 660564936
ground layer bytes: 652000000
rest layers: 8564936


In [96]:
# Layer 0 data 
start_byte_pointer = 96
delta = max_elements_ * size_data_per_element_
data_level0_memory_ = data[start_byte_pointer: start_byte_pointer + delta]
start_byte_pointer += delta
print(start_byte_pointer)

652000096


In [97]:
# Upper layers
links_count = 0
size_links_per_element_ = maxM_ * 4 + 4
element_levels_ = []
links = []
# element_levels_ = np.zeros(max_elements_)
# linkLists_ = [[] for i in range(max_elements_)] 


for i in range(cur_element_count):
    linkListSize =  int.from_bytes(data[start_byte_pointer:start_byte_pointer+4], byteorder='little', signed=False)
    start_byte_pointer += 4
    # if an element is only on ground layer, it has no links on upper layers at all
    if linkListSize == 0:
        element_levels_.append(0)
        links.append([])
#         linkLists_ += None
    # on layer 1: node + link size = 1 * size_links_per_element_
    # on layer 1 & 2: node + link size = 2 * size_links_per_element_
    # ...
    else:
        level = int(linkListSize / size_links_per_element_)
#         print(linkListSize / size_links_per_element_)
        element_levels_.append(level)
        links_tmp = []
        for k in range(int(linkListSize / 4)):
            links_tmp.append(int.from_bytes(data[start_byte_pointer:start_byte_pointer+4], 
                                            byteorder='little', signed=False))
            start_byte_pointer += 4
        links_count += linkListSize / 4;
        links.append(links_tmp)
        
print(links_count)
print(start_byte_pointer)
assert start_byte_pointer == len(data) # 6606296

1141210.0
660564936


Links format 

bytestrings: 0 0 0 ... 

0 -> no upper layer
N -> N bytes for the following string, this is m x 4 x (1 + M), here M = 16 -> 

0 0 0 68 {string contents} 0 0 136 {string contents}

string contents: first element: valid edge number; rest: vector IDs

e.g.,
[
// first 1 + 16 elements
  10,
  
  1561,
  3999,
  4373,
  4213,
  178,
  6898,
  7020,
  7380,
  8454,
  8779,
  3498,
  3755,
  3999,
  4213,
  4373,
  4374,
  
// second 1 + 16 elements
  15,
  
  1191,
  1298,
  1311,
  1781,
  1930,
  2086,
  2598,
  2925,
  2936,
  3262,
  4374,
  4390,
  5441,
  5546,
  5607,
  0],

In [98]:
links

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [16,
  631709,
  206294,
  446625,
  151791,
  47047,
  10945,
  96995,
  333746,
  346449,
  446718,
  243518,
  134522,
  133996,
  963198,
  592566,
  661422],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [14,
  285844,
  320097,
  143916,
  139601,
  134573,
  87381,
  602247,
  423435,
  710798,
  722744,
  129422,
  763931,
  783454,
  989479,
  710798,
  722744],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [16,
  112205,
  66343,
  109404,
  59960,
  65727,
  10536,
  47771,
  190772,
  191803,
  199070,
  247150,
  313122,
  313717,
  316721,
  328332,
  437448],
 [16,
  8974,
  4181,
  303644,
  107023,
  290857,
  26977,
  267549,
  508394,
  731670,
  401555,
  194798,
  588705,
  86466,
  152981,
  25175,
  314291],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [16,
  922073,
  774242,
  229690,
  481152,
  742960,
  670737,
  33666

In [99]:
print("links_count", links_count)
for i in range(30):
    print("{} element_levels_: {}".format(i, element_levels_[i]))

links_count 1141210.0
0 element_levels_: 0
1 element_levels_: 0
2 element_levels_: 0
3 element_levels_: 0
4 element_levels_: 0
5 element_levels_: 0
6 element_levels_: 0
7 element_levels_: 0
8 element_levels_: 0
9 element_levels_: 0
10 element_levels_: 0
11 element_levels_: 1
12 element_levels_: 0
13 element_levels_: 0
14 element_levels_: 0
15 element_levels_: 0
16 element_levels_: 0
17 element_levels_: 0
18 element_levels_: 0
19 element_levels_: 0
20 element_levels_: 0
21 element_levels_: 0
22 element_levels_: 0
23 element_levels_: 0
24 element_levels_: 0
25 element_levels_: 0
26 element_levels_: 0
27 element_levels_: 0
28 element_levels_: 0
29 element_levels_: 1


In [100]:
import struct 

def convertBytes(bytestring, type='int'):
    # bytees = e..g, 4 bytes of data
    if(type=='int'):
        return int.from_bytes(bytestring, byteorder='little', signed=False)
#         return ((b3 & 0xff)<<24) | ((b2 & 0xff)<<16) | ((b1 & 0xff)<<8) | (b0 & 0xff)
    else: # float
        # https://stackoverflow.com/questions/5415/convert-bytes-to-floating-point-numbers
        return struct.unpack('f', bytestring)[0]
#         int8_str = hex (((b3 & 0xff)<<24) | ((b2 & 0xff)<<16) | ((b1 & 0xff)<<8) | (b0 & 0xff))
#         hex2bin = struct.pack('<I', int(int8_str, 0))
#         bin2float = float(struct.unpack('<f', hex2bin)[0])
#         return bin2float


### Wenqi: l0 = Layer 0 (should be L0)
def extractLayer0(data_level0):
    #data_level0 = index_dict["data_level0"]
    size = len(data_level0)
    links_count_l0 = []
    links_l0 = [] 
    data_l0 = []
    label_l0 = []
    label_l0_new = []

    for i in range(size//4):
#         b0 = data_level0[4*i]
#         b1 = data_level0[4*i+1]
#         b2 = data_level0[4*i+2]
#         b3 = data_level0[4*i+3]
        b4 = data_level0[4*i:4*(i+1)]
#         b1 = data_level0[4*i+1]
#         b2 = data_level0[4*i+2]
#         b3 = data_level0[4*i+3]
        j = i % 163 # Wenqi: each elemnt
        if (j==0): # First part: link count of this vector
            links_count_l0.append(convertBytes(b4, type='int'))
#             links_count_l0.append(convertBytes(b0, b1, b2, b3, type='int'))
        elif (j >= 1 and j <= 32): # Second: the IDs of the linked vectors, here 32 = maxM0_ (there are some dummy links)
            links_l0.append(convertBytes(b4, type='int'))  
#             links_l0.append(convertBytes(b0, b1, b2, b3, type='int'))  
        elif (j >= 33 and j <= 160): # Third: 128-D vector
            data_l0.append(convertBytes(b4, type='float'))
#             data_l0.append(convertBytes(b0, b1, b2, b3, type='float'))
        else: 
            continue # skip the vec ID field
#             there are still 8 bytes left: we hereby store 4 bytes at a time, and merge them later
#             label_l0.append(b4)
#             label_l0.append(convertBytes(b0, b1, b2, b3, type='int'))
    
    # vec ID
    for i in range(size//652):
        b8 = data_level0[652*i + 644: 652*i + 652]
        label_l0_new.append(convertBytes(b8, type='int'))
    
    return links_count_l0, links_l0, data_l0, label_l0, label_l0_new

def extractLinks(element_levels, links_list):
    links = []
    count_links = 0
    size = len(element_levels)
    for i in range(size):
        links_tmp = []
        if (element_levels[i]==0):
            links.append(links_tmp)
        else:
            for j in range(element_levels[i]):
                for k in range(17):
#                     b0 = links_list[count_links + 4*k]
#                     b1 = links_list[count_links + 4*k+1]
#                     b2 = links_list[count_links + 4*k+2]
#                     b3 = links_list[count_links + 4*k+3]
                    b4 = links_list[count_links + 4*k: count_links + 4*(k+1)]
                    links_tmp.append(convertBytes(b4, type='int'))  
#                     links_tmp.append(convertBytes(b0, b1, b2, b3, type='int'))  
                count_links += 17 * 4 # 1 for links number, 16 for links
            links.append(links_tmp)
    return links
    
def calculateDist(query_data, currObj, data_l0):
    index = currObj * 128
    currObj_data = data_l0[index:(index+128)]
    query_data = np.array(query_data)
    currObj_data = np.array(currObj_data)
    # return l2 distance between two points
    return np.sum(np.power((query_data-currObj_data), 2))

In [101]:
# data and links in layer0
links_count_l0, links_l0, data_l0, label_l0, label_l0_new = extractLayer0(data_level0_memory_)

In [102]:
print('links_count_l0, len=', len(links_count_l0), links_count_l0[:10])
print('links_l0, len=', len(links_l0), links_l0[:64])
print('data_l0, len=', len(data_l0), data_l0[:128])
print('label_l0, len=', len(label_l0), label_l0[:10])
print('label_l0_new, len=', len(label_l0_new), label_l0_new[:20])

links_count_l0, len= 1000000 [23, 18, 29, 24, 22, 29, 27, 21, 32, 24]
links_l0, len= 32000000 [3271, 594, 433784, 48189, 344037, 10585, 194083, 106116, 35101, 520343, 105411, 209719, 520895, 575383, 95491, 395686, 882992, 2, 648378, 989477, 93937, 226433, 487099, 520895, 570707, 575383, 597767, 648378, 862636, 868303, 882992, 945981, 13370, 780191, 648379, 227346, 518565, 445264, 575384, 395685, 192315, 457580, 148544, 546925, 242702, 338892, 785181, 317121, 862637, 882993, 317121, 441079, 520344, 539319, 546925, 554316, 575384, 591739, 628398, 648379, 768296, 780191, 785181, 794778]
data_l0, len= 128000000 [0.0, 0.0, 0.0, 1.0, 8.0, 7.0, 3.0, 2.0, 5.0, 0.0, 0.0, 3.0, 5.0, 7.0, 11.0, 31.0, 13.0, 0.0, 0.0, 0.0, 0.0, 29.0, 106.0, 107.0, 13.0, 0.0, 0.0, 0.0, 1.0, 61.0, 70.0, 42.0, 0.0, 0.0, 0.0, 0.0, 1.0, 23.0, 28.0, 16.0, 63.0, 4.0, 0.0, 0.0, 0.0, 6.0, 83.0, 81.0, 117.0, 86.0, 25.0, 15.0, 17.0, 50.0, 84.0, 117.0, 31.0, 23.0, 18.0, 35.0, 97.0, 117.0, 49.0, 24.0, 68.0, 27.0, 0.0, 0.0, 0.0, 

In [103]:
sorted_labels = sorted(label_l0_new)
assert sorted_labels == [i for i in range(cur_element_count)]
print(sorted_labels[:10])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [104]:
def mmap_fvecs(fname):
    x = np.memmap(fname, dtype='int32', mode='r')
    d = x[0]
    return x.view('float32').reshape(-1, d + 1)[:, 1:]

def mmap_bvecs(fname):
    x = np.memmap(fname, dtype='uint8', mode='r')
    d = x[:4].view('int32')[0]
    return x.reshape(-1, d + 4)[:, 4:]

def ivecs_read(fname):
    a = np.fromfile(fname, dtype='int32')
    d = a[0]
    # Wenqi: Format of ground truth (for 10000 query vectors):
    #   1000(topK), [1000 ids]
    #   1000(topK), [1000 ids]
    #        ...     ...
    #   1000(topK), [1000 ids]
    # 10000 rows in total, 10000 * 1001 elements, 10000 * 1001 * 4 bytes
    return a.reshape(-1, d + 1)[:, 1:].copy()

def fvecs_read(fname):
    return ivecs_read(fname).view('float32')

In [105]:
dbname = 'SIFT1M'
index_path='../indexes/{}_index.bin'.format(dbname)
dim=128

if dbname.startswith('SIFT'):
    # SIFT1M to SIFT1000M
    dbsize = int(dbname[4:-1])
    xb = mmap_bvecs('/mnt/scratch/wenqi/Faiss_experiments/bigann/bigann_base.bvecs')
    xq = mmap_bvecs('/mnt/scratch/wenqi/Faiss_experiments/bigann/bigann_query.bvecs')
    gt = ivecs_read('/mnt/scratch/wenqi/Faiss_experiments/bigann/gnd/idx_%dM.ivecs' % dbsize)

    N_VEC = int(dbsize * 1000 * 1000)

    # trim xb to correct size
    xb = xb[:dbsize * 1000 * 1000]

    # Wenqi: load xq to main memory and reshape
    xq = xq.astype('float32').copy()
    xq = np.array(xq, dtype=np.float32)
    gt = np.array(gt, dtype=np.int32)

    print("Vector shapes:")
    print("Base vector xb: ", xb.shape)
    print("Query vector xq: ", xq.shape)
    print("Ground truth gt: ", gt.shape)
else:
    print('unknown dataset', dbname, file=sys.stderr)
    sys.exit(1)

Vector shapes:
Base vector xb:  (1000000, 128)
Query vector xq:  (10000, 128)
Ground truth gt:  (10000, 1000)


In [106]:
#### Wenqi: TO Debug 
# (1) the visited list should be a hash set, instead of a bit array

def searchKnn(q_data, k, ep_node, ef, num_elements, max_level, links_count_l0, links_l0, data_l0, links, label_l0_new):
    currObj = ep_node
    curdist = calculateDist(q_data, currObj, data_l0)
    #for layer (1, max_level), find one closest node
    for level in reversed(range(1, max_level+1)):
        print("")
        print("level: ", level)
        changed = True
        while changed:
            print("current object: ", currObj, ", current distance: ", curdist)
            changed = False
            if (len(links[currObj])==0):
                break
            else:
                start_index = (level-1) * 17
                size = links[currObj][start_index]
                print("size of neighbors: ", size) 
                neighbors = links[currObj][(start_index+1):(start_index+17)]
                for i in range(size):
                    cand = neighbors[i]
                    dist = calculateDist(q_data, cand, data_l0)
                    print("cand: ", cand, ", dist: ", dist)
                    if (dist < curdist):
                        curdist = dist
                        currObj = cand
                        changed = True
                        print("changed")
                print("one node finish")
                print("")

    print("")
    print("level: 0")
    # search in ground layer
    visited_array = np.zeros(num_elements, dtype=int) # default 0
    visited_array_tag = 1
    top_candidates = []
    candidate_set = []
    lowerBound = curdist 
    heapq.heappush(top_candidates, (-curdist, currObj))
    heapq.heappush(candidate_set,(curdist, currObj))
    visited_array[currObj] = visited_array_tag

    while len(candidate_set)!=0:
        current_node_pair = candidate_set[0]
        if ((current_node_pair[0] > lowerBound)):
            break
        heapq.heappop(candidate_set)
        current_node_id = current_node_pair[1]
        print("current object: ", current_node_id)
        size = links_count_l0[current_node_id]
        print("size of neighbors: ", size)
        for i in range(size):
            candidate_id = links_l0[32*current_node_id + i]
            if (visited_array[candidate_id]!=visited_array_tag):
                visited_array[candidate_id] = visited_array_tag
                #currObj1 = data_l0[(128*candidate_id):(128*candidate_id+128)]
                dist = calculateDist(q_data, candidate_id, data_l0)
                print("current object: ", candidate_id, ", current distance: ", dist, ", lowerBound: ", lowerBound)
                if (len(top_candidates) < ef or lowerBound > dist):
                    print("added")
                    heapq.heappush(candidate_set, (dist, candidate_id))
                    heapq.heappush(top_candidates, (-dist, candidate_id))
                if (len(top_candidates) > ef):
                    heapq.heappop(top_candidates)
                if (len(top_candidates)!=0):
                    lowerBound = -top_candidates[0][0]
            else :
                print("current object: ", candidate_id, ", visited already")
        print("one node finishes")
        print("")
    
    while len(top_candidates) > k:
        heapq.heappop(top_candidates)

    result = []
    while len(top_candidates) > 0:
        candidate_pair = top_candidates[0]
        result.append([-candidate_pair[0], label_l0_new[candidate_pair[1]]])
        heapq.heappop(top_candidates)

    return result



The parameters in the index header, stored in small endian
uint64_t offsetLevel0_; // 0:8
uint64_t max_elements_; // 8:16
uint64_t cur_element_count; // 16:24
uint64_t size_data_per_element_; // 24:32
uint64_t label_offset_; // 32:40
uint64_t offsetData_; // 40:48
uint32_t maxlevel_; // 48:52
uint32_t enterpoint_node_; // 52:56
uint64_t maxM_; // 56:64
uint64_t maxM0_; // 64:72
uint64_t M_; // 72:80
double mult_; // 80:88
uint64_t ef_construction_; // 88:96

In [109]:
"""
def searchKnn(q_data, k, ep_node, ef, num_elements, max_level, links_count_l0, links_l0, data_l0, links, label_l0_new):
"""
enterpoint_node = enterpoint_node_
ef = 128
num_elements = cur_element_count
max_level = maxlevel_
links = links
result = searchKnn(xq[0], 1, enterpoint_node, ef, num_elements, max_level, links_count_l0, links_l0, data_l0, links, label_l0_new)
print(result)


level:  5
current object:  572337 , current distance:  316481.0
size of neighbors:  0
one node finish


level:  4
current object:  572337 , current distance:  316481.0
size of neighbors:  12
cand:  490454 , dist:  233830.0
changed
cand:  366058 , dist:  217875.0
changed
cand:  81668 , dist:  217828.0
changed
cand:  412162 , dist:  224439.0
cand:  471823 , dist:  348658.0
cand:  174232 , dist:  206661.0
changed
cand:  675393 , dist:  216263.0
cand:  825523 , dist:  272317.0
cand:  860232 , dist:  328888.0
cand:  887321 , dist:  174888.0
changed
cand:  891154 , dist:  247419.0
cand:  983094 , dist:  217262.0
one node finish

current object:  887321 , current distance:  174888.0
size of neighbors:  12
cand:  860232 , dist:  328888.0
cand:  471823 , dist:  348658.0
cand:  572337 , dist:  316481.0
cand:  825523 , dist:  272317.0
cand:  675393 , dist:  216263.0
cand:  412162 , dist:  224439.0
cand:  174232 , dist:  206661.0
cand:  490454 , dist:  233830.0
cand:  366058 , dist:  217875.0
can

current object:  686088 , current distance:  150364.0 , lowerBound:  126411.0
current object:  833562 , current distance:  130552.0 , lowerBound:  126411.0
current object:  834728 , current distance:  170733.0 , lowerBound:  126411.0
current object:  949394 , current distance:  124233.0 , lowerBound:  126411.0
added
current object:  950727 , current distance:  152366.0 , lowerBound:  126153.0
one node finishes

current object:  219536
size of neighbors:  28
current object:  329506 , current distance:  112551.0 , lowerBound:  126153.0
added
current object:  296269 , current distance:  100576.0 , lowerBound:  126106.0
added
current object:  504814 , visited already
current object:  328816 , current distance:  100057.0 , lowerBound:  126099.0
added
current object:  803468 , current distance:  101464.0 , lowerBound:  125928.0
added
current object:  36823 , current distance:  147110.0 , lowerBound:  125916.0
current object:  182954 , current distance:  113867.0 , lowerBound:  125916.0
added

current object:  76575 , current distance:  92329.0 , lowerBound:  110962.0
added
current object:  419707 , current distance:  142936.0 , lowerBound:  110905.0
current object:  429802 , current distance:  122528.0 , lowerBound:  110905.0
current object:  436686 , current distance:  140999.0 , lowerBound:  110905.0
current object:  453650 , current distance:  137837.0 , lowerBound:  110905.0
current object:  525250 , current distance:  152308.0 , lowerBound:  110905.0
current object:  527030 , current distance:  130804.0 , lowerBound:  110905.0
current object:  538943 , current distance:  154716.0 , lowerBound:  110905.0
current object:  587325 , current distance:  151328.0 , lowerBound:  110905.0
current object:  696545 , visited already
current object:  753614 , current distance:  137821.0 , lowerBound:  110905.0
current object:  755952 , current distance:  143033.0 , lowerBound:  110905.0
current object:  812790 , visited already
current object:  920419 , current distance:  137068.0 

current object:  582781 , visited already
current object:  974504 , current distance:  174225.0 , lowerBound:  106609.0
current object:  378760 , current distance:  170174.0 , lowerBound:  106609.0
current object:  360844 , visited already
current object:  910871 , current distance:  147945.0 , lowerBound:  106609.0
current object:  397956 , current distance:  141048.0 , lowerBound:  106609.0
current object:  571948 , current distance:  144839.0 , lowerBound:  106609.0
current object:  104351 , current distance:  132536.0 , lowerBound:  106609.0
current object:  625681 , current distance:  147270.0 , lowerBound:  106609.0
current object:  367571 , current distance:  148982.0 , lowerBound:  106609.0
current object:  274781 , current distance:  144744.0 , lowerBound:  106609.0
current object:  729889 , current distance:  141678.0 , lowerBound:  106609.0
current object:  107322 , current distance:  147165.0 , lowerBound:  106609.0
current object:  17400 , current distance:  151307.0 , low

current object:  614729 , current distance:  164901.0 , lowerBound:  105176.0
current object:  670583 , current distance:  148245.0 , lowerBound:  105176.0
current object:  71955 , current distance:  132379.0 , lowerBound:  105176.0
current object:  8564 , current distance:  176201.0 , lowerBound:  105176.0
current object:  257529 , current distance:  164340.0 , lowerBound:  105176.0
current object:  295678 , current distance:  137803.0 , lowerBound:  105176.0
current object:  770800 , current distance:  143728.0 , lowerBound:  105176.0
current object:  767459 , current distance:  169275.0 , lowerBound:  105176.0
current object:  581100 , current distance:  95995.0 , lowerBound:  105176.0
added
current object:  195984 , current distance:  117295.0 , lowerBound:  105142.0
current object:  200776 , current distance:  117589.0 , lowerBound:  105142.0
current object:  801794 , current distance:  146492.0 , lowerBound:  105142.0
current object:  33630 , current distance:  167081.0 , lowerBo

current object:  16932 , current distance:  123554.0 , lowerBound:  103871.0
current object:  602664 , current distance:  133305.0 , lowerBound:  103871.0
current object:  445157 , current distance:  134610.0 , lowerBound:  103871.0
current object:  15979 , current distance:  130583.0 , lowerBound:  103871.0
current object:  209310 , current distance:  114677.0 , lowerBound:  103871.0
current object:  51632 , current distance:  183770.0 , lowerBound:  103871.0
current object:  35673 , current distance:  147755.0 , lowerBound:  103871.0
current object:  66590 , current distance:  126957.0 , lowerBound:  103871.0
current object:  880978 , current distance:  128950.0 , lowerBound:  103871.0
current object:  71954 , current distance:  125086.0 , lowerBound:  103871.0
current object:  784896 , current distance:  122852.0 , lowerBound:  103871.0
current object:  380366 , current distance:  144171.0 , lowerBound:  103871.0
current object:  35317 , current distance:  110999.0 , lowerBound:  10

current object:  344124 , visited already
current object:  459873 , current distance:  131722.0 , lowerBound:  103496.0
current object:  465030 , current distance:  142558.0 , lowerBound:  103496.0
current object:  604438 , current distance:  127875.0 , lowerBound:  103496.0
current object:  776491 , current distance:  146067.0 , lowerBound:  103496.0
current object:  876550 , current distance:  146487.0 , lowerBound:  103496.0
current object:  888184 , current distance:  112057.0 , lowerBound:  103496.0
current object:  925388 , current distance:  146716.0 , lowerBound:  103496.0
one node finishes

current object:  16367
size of neighbors:  32
current object:  448122 , current distance:  177187.0 , lowerBound:  103496.0
current object:  466427 , current distance:  155189.0 , lowerBound:  103496.0
current object:  972832 , current distance:  154967.0 , lowerBound:  103496.0
current object:  229115 , current distance:  132734.0 , lowerBound:  103496.0
current object:  860303 , current d

In [110]:
print(result)
print(gt[0])

[[61125.0, 504814]]
[504814 344333 900184 657385 205768 278818 831624 885651 504835 504547
 504558 500240 133171 627002 504605 429392 539223 582781 219536 581976
 941875  76575 100277 229024 851705 567738 454435 583350 928756  36011
 372375 296178 582150 283974 209187 893189 372014 582401 233941 576902
 454231 504753 996719 847595 274729 696545  61812 945798 344473 581100
 582620 296136 183643 487010 504502 504484 467203 318561 260404 746178
 296284 504613 371400 582765 847900 505120 813078 331045 582161 544207
 360844 647847 370459 294036 233481 270357 107451 329513  24740 899594
 428858 543901 928802 504487 583065 895937 581853 596184 126826 328816
 504582 462014 581849 928798  66855 301139 720199 296269 648063 292756
 846581 335953 682443 888802 421746 486749 504471 370481 941872 815428
 366950 240216 582631   7397 595040 272693 593131 803468 944727 504772
 295863 602781  23998 810157  99134 276200 426311   8143 191064 495103
 582191 422483 294315 601330 765257   7408 133888 312027 

In [114]:
"""
size = links_count_l0[current_node_id]
print("size of neighbors: ", size)
for i in range(size):
    candidate_id = links_l0[32*current_node_id + i]
"""
for i in range(10):
    size = links_count_l0[i]
    print("count: ", size)
    for j in range(size):
        candidate_id = links_l0[32*i + j]
        print(candidate_id)

count:  23
3271
594
433784
48189
344037
10585
194083
106116
35101
520343
105411
209719
520895
575383
95491
395686
882992
2
648378
989477
93937
226433
487099
count:  18
13370
780191
648379
227346
518565
445264
575384
395685
192315
457580
148544
546925
242702
338892
785181
317121
862637
882993
count:  29
185652
151195
371206
477244
104259
225588
520092
95720
438617
546734
102463
550479
72540
318697
437073
609273
134271
696789
627431
134681
0
715517
157803
783439
797411
896320
910395
913210
913888
count:  24
4976
7133
52128
62904
12632
178405
237426
454545
43797
247022
131829
172501
126136
87680
255303
237220
445505
544394
237355
6080
589332
613056
824145
983596
count:  22
85032
23001
22198
243518
372541
597359
11017
443003
201399
570111
112998
683769
93565
106415
473139
713323
765628
796818
802184
882994
894127
924447
count:  29
304474
11544
416718
141137
210849
74433
405734
326980
3521
211748
112994
628143
57543
106266
415126
497318
302966
386773
921926
954997
211810
541409
291907
59616

In [120]:
node_id = 999998
size = links_count_l0[node_id]
print("count: ", size)
for j in range(32):
    print(links_l0[32 * node_id + j])

count:  16
280941
582127
537227
119174
297369
821206
436245
131177
393308
819524
408415
462517
836599
202007
786426
92219
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [105]:
print(len(links_count_l0), np.min(links_count_l0), np.max(links_count_l0))
print("\n==== first links ====\n")
for i in range(10):
    print(links_count_l0[i])
print("\n=== last links ====\n")
for i in range(10):
    print(links_count_l0[-i])

1000000 1 32

==== first links ====

23
18
29
24
22
29
27
21
32
24

=== last links ====

23
5
16
13
10
16
14
11
16
16


In [103]:
print("len: ", len(links_l0))
print("\n==== first links ====\n")
for i in range(1000):
    print(links_l0[i])
    
print("\n=== last links ====\n")
for i in range(128):
    print(links_l0[-i])

len:  32000000

==== first links ====

3271
594
433784
48189
344037
10585
194083
106116
35101
520343
105411
209719
520895
575383
95491
395686
882992
2
648378
989477
93937
226433
487099
520895
570707
575383
597767
648378
862636
868303
882992
945981
13370
780191
648379
227346
518565
445264
575384
395685
192315
457580
148544
546925
242702
338892
785181
317121
862637
882993
317121
441079
520344
539319
546925
554316
575384
591739
628398
648379
768296
780191
785181
794778
185652
151195
371206
477244
104259
225588
520092
95720
438617
546734
102463
550479
72540
318697
437073
609273
134271
696789
627431
134681
0
715517
157803
783439
797411
896320
910395
913210
913888
661417
684523
696789
4976
7133
52128
62904
12632
178405
237426
454545
43797
247022
131829
172501
126136
87680
255303
237220
445505
544394
237355
6080
589332
613056
824145
983596
237355
237426
247022
255303
336338
445505
454545
475098
85032
23001
22198
243518
372541
597359
11017
443003
201399
570111
112998
683769
93565
106415
473139

In [170]:
# Save link to txt for graph partition
num_vertices = len(links_count_l0)
num_edges = int(np.sum(links_count_l0) / 2)
print("num_vertices", num_vertices)
print("num_edges", num_edges)

lines = []
lines.append(str(num_vertices) + " " + str(num_edges) + " 11\n")
for node_id in range(num_vertices):
    size = links_count_l0[node_id]
    s = "1 "
    for j in range(size):
        if node_id == links_l0[32 * node_id + j]:
            print('self loop! ', links_l0[32 * node_id + j])
#         if node_id > links_l0[32 * node_id + j]: # only count each edge once
#             continue
#         else:
        s += str(links_l0[32 * node_id + j] + 1) + " 1 " # KHip count from node ID = 1, so + 1 here
    s = s[:-1] # remove the last " "
    s += "\n"
    lines.append(s)

num_vertices 10000
num_edges 97929


In [171]:
for i in range(5):
    print(lines[i])

10000 97929 11

1 10 1 3272 1 24 1 3598 1 4429 1 3601 1 2822 1 2781 1 870 1 595 1 1020 1 1989 1 9 1 5345 1 5344 1 5399 1 5580 1 5965 1 5970 1 6018 1 6464 1 6735 1 6907 1 7201 1

1 862 1 1534 1 1904 1 691 1 421 1 2198 1 407 1 2330 1 734 1 410 1 396 1 871 1 453 1 441 1 546 1 2867 1 2870 1 2892 1 2899 1 2972 1 3047 1 3154 1 3721 1 4295 1 4666 1 4876 1 5216 1 5315 1 5686 1 5794 1 6019 1

1 53 1 904 1 2346 1 3195 1 218 1 2485 1 3623 1 944 1 995 1 472 1 125 1 2577 1 378 1 3645 1 3771 1 4285 1 4432 1 4769 1 4952 1 4977 1 5384 1 5494 1 5767 1 6081 1 7134 1 8394 1 8398 1 9776 1 9887 1

1 273 1 346 1 420 1 6 1 107 1 31 1 5 1 449 1 682 1 798 1 909 1 921 1 933 1 1044 1 1333 1 1523 1 2861 1 3036 1 3103 1 3433 1 3992 1 3994 1 4956 1 5213 1 5276 1 5744 1 7776 1 8122 1 9682 1 9952 1



In [173]:
graph_file = '../indexes/SIFT10K_ground.graph'
with open(graph_file, 'w') as f:
    f.writelines(lines)

In [61]:
# links in other layers
links = extractLinks(element_levels_, linkLists_)

In [72]:
import pickle
import os

def save_obj(obj, dirc, name):
    # note use "dir/" in dirc
    with open(os.path.join(dirc, name + '.pkl'), 'wb') as f:
        pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL) # for py37,pickle.HIGHEST_PROTOCOL=4

save_obj(links, '../indexes', 'SIFT1M_upper_layer_links')

In [68]:
print(len(links))
for i in range(20):
    print(links[i])

1000000
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[16, 631709, 206294, 446625, 151791, 47047, 10945, 96995, 333746, 346449, 446718, 243518, 134522, 133996, 963198, 592566, 661422]
[]
[]
[]
[]
[]
[]
[]
[]


In [113]:
links[151791]

[15,
 299015,
 227593,
 436393,
 309082,
 11,
 15121,
 446393,
 464062,
 137892,
 126304,
 365733,
 498339,
 956340,
 983443,
 989396,
 455024]

In [70]:
import kahip;

#build adjacency array representation of the graph

# Wenqi: node 0 -> start from position 0 of the list below
# Wenqi: node 1 -> start from position 2 of the list below
# Wenqi: dummy node -> start from 12 -> out of range of the adjncy list; 
xadj           = [0,2,5,7,9,12]; # Wenqi: node 0 -> start from position 0 of the list below
adjncy         = [1,4,0,2,4,1,3,2,4,0,1,3];
vwgt           = [1,1,1,1,1]
adjcwgt        = [1,1,1,1,1,1,1,1,1,1,1,1]
print("length of xadj: {}\tadjncy: {}\tvwg: {}\tadjcwgt: {}".format(len(xadj), len(adjncy), len(vwgt), len(adjcwgt)))
supress_output = 0
imbalance      = 0.03
nblocks        = 3 
seed           = 0

# set mode 
#const int FAST           = 0;
#const int ECO            = 1;
#const int STRONG         = 2;
#const int FASTSOCIAL     = 3;
#const int ECOSOCIAL      = 4;
#const int STRONGSOCIAL   = 5;
mode = 2 

edgecut, blocks = kahip.kaffpa(vwgt, xadj, adjcwgt, 
                              adjncy,  nblocks, imbalance, 
                              supress_output, seed, mode)

print("Edge cut", edgecut)
print("Blocks", blocks)

ModuleNotFoundError: No module named 'kahip'

In [13]:
dim=128
p = hnswlib.Index(space='l2', dim=dim)  # the space can be changed - keeps the data, alters the distance function.

print("\nLoading index from {}\n".format(index_path))

# Increase the total capacity (max_elements), so that it will handle the new data
p.load_index(index_path)
# p.load_index(index_path, max_elements=N_VEC)

index = p.annData()


Loading index from ../indexes/SIFT1M_index.bin



AttributeError: 'hnswlib.Index' object has no attribute 'annData'

In [3]:
dim = 128
num_elements = 1048576
index_path = 'index.bin'

p = hnswlib.Index(space = 'l2', dim = dim)
p.load_index(index_path)
index = p.annData()
index_dict = index[0]

# ef = index_dict["ef"]
ef = 16
max_level = index_dict["max_level"]
data_level0 = index_dict["data_level0"]
enterpoint_node = index_dict["enterpoint_node"]
element_levels = index_dict["element_levels"]
links_list = index_dict["link_lists"]

# data and links in layer0
links_count_l0, links_l0, data_l0, label_l0_new = extractLayer0(data_level0)
# links in other layers
links = extractLinks(element_levels, links_list)

# np.random.seed(1)
# qdata = np.float32(np.random.random((1, dim)))
qdata = [0.0123696, 0.472579,  0.586607, 0.15996, 0.337809, 0.380651, 0.351833, 0.59557, 0.523551, 0.501098, 0.267685, 0.802518, 0.977961, 0.767567, 0.0956629, 0.97753,
        0.67074,   0.0839634, 0.969068, 0.617426, 0.249031, 0.224536, 0.519282, 0.943601, 0.0505886, 0.452829, 0.0905339, 0.413765, 0.136365, 0.8277, 0.309194, 0.766323,
        0.113263,  0.960899,  0.560823, 0.236987, 0.802165, 0.577352, 0.242747, 0.661184, 0.63294, 0.961649, 0.718534, 0.664026, 0.311829, 0.776273, 0.0870592, 0.117126,
        0.180718,  0.606929,  0.191287, 0.000166441, 0.582858, 0.28387, 0.638824, 0.479961, 0.86328, 0.0235504, 0.567014, 0.474131, 0.67512, 0.569185, 0.95862, 0.251585,
        0.673246,  0.793391,  0.524749, 0.280569, 0.351585, 0.454706, 0.108521, 0.318753, 0.0697084, 0.45125, 0.238437, 0.567582, 0.865705, 0.198373, 0.380745, 0.131855,
        0.486811,  0.468224,  0.479084, 0.987974, 0.023348, 0.34968, 0.654999, 0.194506, 0.716907, 0.127722, 0.450073, 0.668612, 0.116613, 0.918388, 0.353905, 0.878461,
        0.728848,  0.174207,  0.926245, 0.315212, 0.489977, 0.718842, 0.928661, 0.315497, 0.875097, 0.146088, 0.881823, 0.80182, 0.327934, 0.288367, 0.751575, 0.323576,
        0.746039,  0.658331,  0.597801, 0.282286, 0.643329, 0.518495, 0.027351, 0.911921, 0.488524, 0.877867, 0.168668, 0.783541, 0.954007, 0.337684, 0.604556, 0.0190392]
qdata = np.asarray(qdata, dtype=float)
qdata = np.reshape(qdata, (-1, 128))

result = searchKnn(qdata, 1, enterpoint_node, ef, num_elements, max_level, links_count_l0, links_l0, data_l0, links, label_l0_new)


level:  2
current object:  115 , current distance:  26.52131756653322
size of neighbors:  0
one node finish


level:  1
current object:  115 , current distance:  26.52131756653322
size of neighbors:  13
cand:  11 , dist:  20.059007862703073
changed
cand:  38 , dist:  20.790651997941488
cand:  59 , dist:  20.91612832401094
cand:  39 , dist:  18.47143744246125
changed
cand:  29 , dist:  22.154903538423802
cand:  84 , dist:  24.08333735526193
cand:  128 , dist:  21.112510169232273
cand:  159 , dist:  20.76760140538351
cand:  165 , dist:  19.8821124428234
cand:  173 , dist:  19.64231683231375
cand:  174 , dist:  22.49861610761772
cand:  177 , dist:  21.14078619581963
cand:  183 , dist:  19.00116690205011
one node finish

current object:  39 , current distance:  18.47143744246125
size of neighbors:  13
cand:  11 , dist:  20.059007862703073
cand:  29 , dist:  22.154903538423802
cand:  38 , dist:  20.790651997941488
cand:  59 , dist:  20.91612832401094
cand:  84 , dist:  24.08333735526193
ca

In [4]:
print(links_l0[(2*32):(2*32+32)])

[1, 0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 20, 21, 25, 30, 32, 35, 37, 39, 43, 62, 81, 127, 128, 134, 149, 173, 181]


In [5]:
result

[[0.07357056635117745, 114]]